# smolagents

In [1]:
import datasets
from langchain_core.documents import Document

guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")

docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest['name']}
    )
    for guest in guest_dataset
]

README.md:   0%|          | 0.00/371 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/3.32k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3 [00:00<?, ? examples/s]

In [4]:
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever

class GuestInfoRetrieverTool(Tool):
    name = "guest_info_retriever"
    description = "Retrieves detailed information about gala guests based on their name or relation."
    inputs = {
        "query": {
            "type": "string",
            "description": "The name or relation of the guest you want information about."
        }
    }
    output_type = "string"

    def __init__(self, docs, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.is_initialized = False
        self.retriever = BM25Retriever.from_documents(docs)

    def forward(self, query: str):
        results = self.retriever.get_relevant_documents(query)
        if results:
            return "\n\n".join([doc.page_content for doc in results[:3]])
        else:
            return "No matching guest information found"

guest_info_tool = GuestInfoRetrieverTool(docs)


In [5]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel()

alfred = CodeAgent(tools=[guest_info_tool], model=model)

response = alfred.run("Tell me about our guest named 'Lady Ada Lovelace'")

print("Alfred's Response")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about our guest named 'Lady Ada Lovelace'                                                               │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  ada_info = guest_info_retriever(query="Lady Ada Lovelace")                                                       
  print(ada_info)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

/var/folders/56/_q1k7zs52t9997xq9ncxwdx00000gn/T/ipykernel_97665/180376248.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = self.retriever.get_relevant_documents(query)


Execution logs:
Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Lady Ada Lovelace is an esteemed mathematician and friend, renowned for her pioneering work in     
  mathematics and computing. She is celebrated as the first computer programmer due to her work on Charles         
  Babbage's Analytical Engine. Her email is ada.lovelace@example. com.")                                           
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Lady Ada Lovelace is an esteemed mathematician and friend, renowned for her pioneering work in 
mathematics and computing. She is celebrated as the first computer programmer due to her work on Charles Babbage's 
Analytical Engine. Her email is ada.lovelace@example. com.

[Step 2: Duration 1.57 seconds| Input tokens: 4,504 | Output tokens: 156]

Alfred's Response
Lady Ada Lovelace is an esteemed mathematician and friend, renowned for her pioneering work in mathematics and computing. She is celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine. Her email is ada.lovelace@example. com.


In [6]:
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()

results = search_tool("Who's the current President of France?")
print(results)

## Search Results

[President of France - Wikipedia](https://en.wikipedia.org/wiki/President_of_France)
The president of France is the ex officio co-prince of Andorra, grand master of the Legion of Honour and of the National Order of Merit, and protector of the Institut de France in Paris.

[Presidents of the Republic | Élysée](https://www.elysee.fr/en/french-presidency/the-presidents-of-the-republic)
Discover the biographies of the twenty-five people who served as President of the French Republic, from the 2nd to the 5th Republic.Yesterday, 150th anniversary of the Republic. The Presidents since 1848.

[With Sébastien Lecornu’ s Resignation, France Faces Increasing Turmoil](https://www.nytimes.com/2025/10/07/world/europe/france-government-turmoil-sebastien-lecornu.html)
As France Faces Political Turmoil, Macron Is Backed Into a Corner. The record-fast collapse of yet another government confronts President Emmanuel Macron and his country with an intensifying crisis.

[‘Lame Duck’ Macro

In [7]:
from smolagents import Tool
import random

class WeatherInfoTool(Tool):
    name = "weather_info"
    description = "Fetches dummy weather information for a given location."
    inputs = {
        "location": {
            "type": "string",
            "description": "The location to get weather information for."
        }
    }
    output_type = "string"

    def forward(self, location: str):
        # Dummy weather data
        weather_conditions = [
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Clear", "temp_c": 25},
            {"condition": "Windy", "temp_c": 20}
        ]
        # Randomly select a weather condition
        data = random.choice(weather_conditions)
        return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = WeatherInfoTool()

In [8]:
from smolagents import Tool
from huggingface_hub import list_models

class HubStatsTool(Tool):
    name = "hub_stats"
    description = "Fetches the most downloaded model from a specific author on the Hugging Face Hub."
    inputs = {
        "author": {
            "type": "string",
            "description": "The username of the model author/organization to find models from."
        }
    }
    output_type = "string"

    def forward(self, author: str):
        try:
            # List models from the specified author, sorted by downloads
            models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

            if models:
                model = models[0]
                return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
            else:
                return f"No models found for author {author}."
        except Exception as e:
            return f"Error fetching models for {author}: {str(e)}"

# Initialize the tool
hub_stats_tool = HubStatsTool()

# Example usage
print(hub_stats_tool("facebook")) # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/contriever with 8,210,171 downloads.


In [11]:
from smolagents import CodeAgent, InferenceClientModel, LiteLLMModel

# Initialize the Hugging Face model
# model = InferenceClientModel()
model = LiteLLMModel(
    model_id="ollama_chat/llama3.1",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192,
)


# Create Alfred with all the tools
alfred = CodeAgent(
    tools=[search_tool, weather_info_tool, hub_stats_tool],
    model=model
)

# Example query Alfred might receive during the gala
response = alfred.run("What is Facebook and what's their most popular model?")

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Facebook and what's their most popular model?                                                           │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.1 ───────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Get some background on Facebook                                                                                
  facebook_background = web_search(query="Facebook")                                                               
  print(facebook_background)                                                                                       
                                                                                                                   
  # Get information about Facebook's models on Hugging Face Hub                                                    
  author = "facebook"                                                                                              
  most_downloaded_model = hub_stats(author=author)                                                                 
  print(most_downloaded_model)                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Facebook](https://en.wikipedia.org/wiki/Facebook)
Facebook is an American social media and social networking service owned by the American technology conglomerate 
Meta. Created in 2004 by Mark Zuckerberg with four other Harvard College students and roommates, Eduardo Saverin, 
Andrew McCollum, Dustin Moskovitz, and Chris Hughes, its name derives from the face book directories often given to
American university students. Membership was initially limited to Harvard students, gradually expanding to other 
North American universities. Since 2006, Facebook allows everyone to register from 13 years old, except in the case
of a handful of nations, where the age requirement is 14 years. As of December 2023, Facebook claimed almost 3.07 
billion monthly active users worldwide. As of July 2025, Facebook ranked as the third-most-visited website in the 
world, with 23% of its traffic coming from the United States. It was the most downloaded mobile app of the 
2010s.Facebook can be accessed from devices with Internet connectivity, such as personal computers, tablets and 
smartphones. After registering, users can create a profile revealing personal information about themselves. They 
can post text, photos and multimedia which are shared with any other users who have agreed to be their friend or, 
with different privacy settings, publicly. Users can also communicate directly with each other with Messenger, edit
messages (within 15 minutes after sending), join common-interest groups, and receive notifications on the 
activities of their Facebook friends and the pages they follow.Facebook has often been criticized over issues such 
as user privacy (as with the Facebook–Cambridge Analytica data scandal), political manipulation (as with the 2016 
U.S. elections) and mass surveillance. The company has also been subject to criticism over its psychological 
effects such as addiction and low self-esteem, and over content such as fake news, conspiracy theories, copyright 
infringement, and hate speech. Commentators have accused Facebook of willingly facilitating the spread of such 
content, as well as exaggerating its number of users to appeal to advertisers.

[Facebook is turning into TikTok | The 
Verge](https://www.theverge.com/news/794923/meta-facebook-tiktok-reels-friend-bubble)
Meta’s work to improve its ranking systems has already increased time spent on videos on Facebook by more than 20 
percent year-over-year, the company revealed in its latest earnings report.

[' Facebook is spying on me': User gets ads for obscure... - The 
Mirror](https://www.mirror.co.uk/news/uk-news/facebook-isnt-spying-us-ads-12362519)
Facebook CEO Mark Zuckerberg has denied the social network uses your phone's microphone to listen to your 
conversations and target you with ads...

[Is Facebook a monopoly? Mark Zuckerberg doesn’t have an 
answer](https://qz.com/1249296/is-facebook-a-monopoly-facebook-ceo-zuckerberg-didnt-have-an-answer-in-congressional
)
The Facebook CEO weaved around the question, citing a statistic that “the average American uses eight different 
apps to communicate with their friends and stay in touch.”

[Facebook updates its algorithm to give users more... | 
TechCrunch](https://techcrunch.com/2025/10/07/facebook-updates-its-algorithm-to-give-users-more-control-over-which-
videos-they-see/)
Facebook Reels will also feature AI-powered search suggestions. These suggestions will recommend queries to help 
users discover more content about topics they like, similar to the gray...

[Facebook Ads for SaaS: Why Most B2B Companies Get... - Aimers 
Blog](https://aimers.io/blog/facebook-ads-for-saas-why-most-b2b-companies-get-it-wrong)
Facebook ads work for SaaS companies. We've seen them drive 4X increases in sales opportunities for companies like 
Orion Labs.

[Facebook Statistics 2025: Key Trends and User Insights](https://www.twinstrata.com/facebook-statistics/)
This article dives deep into the latest Facebook stati

[Step 1: Duration 19.97 seconds| Input tokens: 2,136 | Output tokens: 110]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Final Answer                                                                                                   
  final_answer("Facebook is an American social media and social networking service owned by the American           
  technology conglomerate Meta, with almost 3.07 billion monthly active users worldwide. The most downloaded       
  model by Facebook is facebook/contriever with 8,210,171 downloads.")                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Facebook is an American social media and social networking service owned by the American technology 
conglomerate Meta, with almost 3.07 billion monthly active users worldwide. The most downloaded model by Facebook 
is facebook/contriever with 8,210,171 downloads.

[Step 2: Duration 11.12 seconds| Input tokens: 5,543 | Output tokens: 200]

🎩 Alfred's Response:
Facebook is an American social media and social networking service owned by the American technology conglomerate Meta, with almost 3.07 billion monthly active users worldwide. The most downloaded model by Facebook is facebook/contriever with 8,210,171 downloads.


In [19]:
# Initialize the Hugging Face model
# model = InferenceClientModel()
model = LiteLLMModel(
    model_id="ollama_chat/llama3.1",  # Or try other Ollama-supported models
    api_base="http://127.0.0.1:11434",  # Default Ollama local server
    num_ctx=8192,
)
# Initialize the web search tool
search_tool = DuckDuckGoSearchTool()

# Initialize the weather tool
weather_info_tool = WeatherInfoTool()

# Initialize the Hub stats tool
hub_stats_tool = HubStatsTool()

# Create Alfred with all the tools
alfred = CodeAgent(
    tools=[guest_info_tool, weather_info_tool, hub_stats_tool, search_tool],
    model=model,
    add_base_tools=True,  # Add any additional base tools
    planning_interval=3,   # Enable planning every 3 steps
    max_steps=10
)

In [ ]:
query = "Tell me about gala guest info named 'Lady Ada Lovelace'"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

In [20]:
query = "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather like in Paris tonight? Will it be suitable for our fireworks display?                        │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/llama3.1 ───────────────────────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## Part 1: Facts Survey

### 1.1. Facts given in the task
* None

### 1.2. Facts to look up
* Weather information for Paris tonight
        + Location: Paris
        + Time: Tonight (exact time not specified, but can be approximated)
        + Source: Current weather APIs or websites (e.g., AccuWeather, Weather.com)

### 1.3. Facts to derive
* None

## Part 2: Plan

1. **Search for current Paris weather**: Use the `web_search` function with a query like "Paris weather tonight" to
find reliable sources of information.
2. **Determine suitable time and location**: From the search results, identify the current time in Paris (e.g., 
UTC+1) and confirm that we are referring to the correct location.
3. **Fetch weather API data**: Utilize an online weather API (e.g., OpenWeatherMap, Dark Sky) or a website like 
AccuWeather to retrieve up-to-date weather information for Paris at the current time.
4. **Evaluate weather conditions**: Analyze the retrieved weather data to assess whether it will be suitable for a 
fireworks display (e.g., clear skies, moderate temperatures).
5. **Final assessment and recommendation**: Based on the evaluation, provide a final answer regarding the 
suitability of the weather for the fireworks display.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = web_search(query="Paris weather tonight")                                                              
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[AccuWeather Paris, Ville de Paris, France Weather Forecast | 
AccuWeather](https://www.accuweather.com/en/fr/paris/623/weather-forecast/623)
Paris, Ville de Paris Weather Today ... Monthly Air Quality Health & Activities · Tue, Oct 7 · Clouds yielding to 
sun Hi: 67° · Tonight: Increasing clouds Lo: 52° ·...

[10-Day Weather Forecast for Paris, France - The Weather 
...](https://weather.com/weather/tenday/l/1a8af5b9d8971c46dd5a52547f9221e22cd895d8d8639267a87df614d0912830)
Be prepared with the most accurate 10-day forecast for Paris , France with highs, lows, chance of precipitation 
from The Weather Channel and Weather .com

[Today's Weather in Paris - Hourly Forecast and 
Conditions](https://www.easeweather.com/europe/france/ile-de-france/paris/today)
1 day ago · Get the latest hourly weather updates for Paris today. Detailed forecast including temperature, wind, 
rain, snow, and UV index. Stay informed about today's weather conditions in Paris.

[Paris Weather. 14 Day Forecast - 
WorldWeatherOnline.com](https://www.worldweatheronline.com/en-fr/paris-weather/ile-de-france/fr.aspx)
Summers in Paris are typically warm and pleasant, with temperatures averaging between 18°C to 25°C, making it ideal
for leisurely strolls along the Seine as the city basks under generous sunshine. However, occasional rain showers 
might surprise you, so keeping an umbrella handy is a good idea.

[Hourly forecast for Paris, Paris, France - 
timeanddate.com](https://www.timeanddate.com/weather/france/paris/hourly)
Sep 23, 2025 · Currently: 64 °F. Passing clouds. (Weather station: Villacoublay, France). See more current weather

[Weather Paris - meteoblue](https://www.meteoblue.com/en/weather/week/paris_france_2988507)
Today's and tonight 's professional weather forecast for Paris. Precipitation radar, HD satellite images, and 
current weather warnings, hourly temperature, chance of rain, and sunshine hours.

[Paris, IDF, FR Hourly Forecast - The Weather 
Network](https://www.theweathernetwork.com/en/city/fr/ile-de-france/paris/hourly)
Get Paris, IDF, FR current weather report with temperature, feels like, wind, humidity, pressure, UV and more from 
TheWeatherNetwork.com.

[BBC Paris - BBC Weather](https://www.bbc.com/weather/2988507)
Partly cloudy and light winds · Humidity · 94% Pressure · 1022 mb · Visibility · Moderate · Temperature feels like 
13°55° · Precipitation is not expected · Light winds from the north north east · See more weather for · Wednesday 
8th · Last updated today at 13:00 ·

[Met Office Paris (France) weather - Met Office](https://weather.metoffice.gov.uk/forecast/u09tvnxyj)
Paris 7 day weather forecast including weather warnings, temperature, rain, wind, visibility, humidity and UV

[Weather Underground Paris, France 10-Day Weather Forecast | Weather 
Underground](https://www.wunderground.com/forecast/fr/paris)
Paris Weather Forecasts. Weather Underground provides local & long-range weather forecasts, weatherreports, maps & 
tropical weather conditions for the Paris area.

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  results = web_search(query="Paris weather forecast tonight")                                                     
  print(results)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[What are your food “must-haves” in Paris? : 
r/ParisTravelGuide](https://www.reddit.com/r/ParisTravelGuide/comments/1c8pt3p/what_are_your_food_musthaves_in_pari
s/)
A community for Paris lovers or first-timers about traveling to the city, as a tourist or off the beaten path. 
Itinerary reviews, trip reports, advice from local or experienced travelers, niche questions, …

[The Paris…. Is it good? : r/LasVegas - 
Reddit](https://www.reddit.com/r/LasVegas/comments/12jze29/the_paris_is_it_good/)
For the money, Paris is one of the best on the strip. Location is amazing, Rooms/Pool are good, casino is pretty, 
connects to the Horsehoe, CVS right there is clutch.

[is paris as bad as reddit describes? : 
r/travel](https://www.reddit.com/r/travel/comments/16mny1v/is_paris_as_bad_as_reddit_describes/)
14 votes, 101 comments. I'm doing a backpacking trip of many european countries and of course, paris is on my 
itenirary. But, all over the internet I…

[Paris et sa banlieue - Reddit](https://www.reddit.com/r/paris/)
A Paris on a les monuments historiques, le PLU, les soit-disant toits en zinc qu'on veut classer au patrimoine 
mondial de l'UNESCO, les règlements de copropriété qui peuvent empêcher d'installer …

[Great restaurants in Paris: list of personal favs that people ... - 
Reddit](https://www.reddit.com/r/paris/comments/zktov3/great_restaurants_in_paris_list_of_personal_favs/)
Dec 13, 2022 · Great restaurants in Paris: list of personal favs that people strongly recommend (compilation of 
places mentioned in recent Paristravelguide post). Do the places in this list get …

[Paris Travel Guide - Reddit](https://www.reddit.com/r/ParisTravelGuide/wiki/index/)
A community for Paris lovers or first-timers about traveling to the city, as a tourist or off the beaten path. 
Itinerary reviews, trip reports, advice from local or experienced travelers, niche questions, …

[Paris is dirty? : r/ParisTravelGuide - 
Reddit](https://www.reddit.com/r/ParisTravelGuide/comments/184kp2u/paris_is_dirty/)
A community for Paris lovers or first-timers about traveling to the city, as a tourist or off the beaten path. 
Itinerary reviews, trip reports, advice from local or experienced travelers, niche questions, …

[Paris hotel : r/vegas - Reddit](https://www.reddit.com/r/vegas/comments/1bxc9ub/paris_hotel/)
Paris is personally not my taste, but compared to other CET hotels, it's completely fine. The sportsbook has 
recently been redone, beer park is nice, mon ami gabi is a great brunch spot and …

[My Paris restaurant recommendations : r/ParisTravelGuide - 
Reddit](https://www.reddit.com/r/ParisTravelGuide/comments/1db8oj9/my_paris_restaurant_recommendations/)
Hey everyone! Just spent 10 days in Paris hitting a lot of great restaurants and would love to pass on my 
favourites in case someone wants some recommendations. Le Matré French food / $$ / …

[[MEGA LIST] All the coords you'll ever need! - 
Reddit](https://www.reddit.com/r/PokemonGoSpoofing/comments/ehdcp0/mega_list_all_the_coords_youll_ever_need/)
Dec 29, 2019 · People are always looking for good coords so here are 50 of my favorite locations. For iSpoofer 
users you can copy the list (see the first reply) and import it directly into iSpoofer. …

Out: None

[Step 10: Duration 47.75 seconds| Input tokens: 60,209 | Output tokens: 836]

KeyboardInterrupt: 

In [ ]:
query = "One of our guests is from Qwen. What can you tell me about their most popular model?"
response = alfred.run(query)

print("🎩 Alfred's Response:")
print(response)

In [ ]:
# Create Alfred with conversation memory
alfred_with_memory = CodeAgent(
    tools=[guest_info_tool, weather_info_tool, hub_stats_tool, search_tool],
    model=model,
    add_base_tools=True,
    planning_interval=3
)

# First interaction
response1 = alfred_with_memory.run("Tell me about Lady Ada Lovelace.")
print("🎩 Alfred's First Response:")
print(response1)

# Second interaction (referencing the first)
response2 = alfred_with_memory.run("What projects is she currently working on?", reset=False)
print("🎩 Alfred's Second Response:")
print(response2)